In [1]:
# To use TF 2.0 (on EC2 instance running Deep Learning AMI):
# source activate tensorflow_p36
# pip uninstall tensorflow-gpu
# pip install tensorflow-gpu==2.0.0-alpha0

# But then later decided wanted even newer GPU stuff and in that conda env, ran:
# pip uninstall tensorflow-gpu
# pip install --upgrade pip
# pip install wrapt --ignore-installed # ran this because had an error
# pip install  tf-nightly-gpu-2.0-preview

# Result: Successfully installed tf-nightly-gpu-2.0-preview-2.0.0.dev20190527

In [2]:
import os
from datetime import datetime
import numpy as np
import tensorflow as tf

In [3]:
from packaging import version

print("TensorFlow version: ", tf.__version__) # make sure >= 2.0.0-dev20190527
assert version.parse(tf.__version__).release[0] >= 2, "This notebook requires TensorFlow 2.0 or above."

TensorFlow version:  2.0.0-dev20190527


In [4]:
import cs230_project_utilities as utils

In [5]:
# GPU usage logging (TF 2.0+)

tf.config.set_soft_device_placement(True)
tf.debugging.set_log_device_placement(False)

# Prepare CIFAR-100 dataset

In [6]:
# Load cifar 100 (should already be shuffled)
(x_train, labels_train), (x_test, labels_test) = tf.keras.datasets.cifar100.load_data(label_mode='fine')

# Convert x_train to float32, grayscale
x_train = x_train.astype('float32')
x_train =(0.299 * x_train[..., 0] + 0.587 * x_train[..., 1] +  0.114 * x_train[..., 2]) / 255.0

# Convert x_test to float32, grayscale 
x_test = x_test.astype('float32')
x_test =(0.299 * x_test[..., 0] + 0.587 * x_test[..., 1] +  0.114 * x_test[..., 2]) / 255.0

# Split x_test to create x_dev
x_dev, x_test = x_test[:len(x_test) // 2], x_test[len(x_test) // 2:]

# Show stats of images
print('Shape of x_train: ' + str(x_train.shape))
print('Shape of x_test: ' + str(x_test.shape))
print('Shape of x_dev: ' + str(x_dev.shape))

Shape of x_train: (50000, 32, 32)
Shape of x_test: (5000, 32, 32)
Shape of x_dev: (5000, 32, 32)


In [7]:
def fft_2d_scaled_and_centered(tensor):
    ''' 
    Input
        tensor: 2D tensor of shape [height, width]
    Returns
        fft: centered_2d_fft(tensor) / sqrt(product(tensor.shape)) as dtype tf.complex64
        
    Tested and np.allclose(fft, numpy_fft, atol=1e-5) returns True, where
    numpy_fft = np.abs(utils.signal_processing.fft_2D_centered(tensor)).
    
    '''
    complex_valued = tf.cast(tf.squeeze(tensor), tf.complex64)
    fft = tf.signal.fft2d(complex_valued)
    scale = tf.math.sqrt(tf.reduce_prod(tf.cast(fft.shape, tf.float32)))
    fft_scaled = fft / tf.cast(scale, tf.complex64)
    centered_fft = tf.signal.fftshift(fft_scaled)
    return centered_fft

def cifar_parser(sample):
    # Returns: (fft, image reconstruction) pairs for automap model
    
    # Image must be 3-dim
    sample = tf.expand_dims(sample, -1)
    resized = tf.image.resize(sample, [256, 256])
    fft = fft_2d_scaled_and_centered(tf.squeeze(resized))
    fft = tf.expand_dims(fft, -1) # tf.signal.fft2d expects 2D input, so we undo the squeeze() from before
    
    # Separate magnitude and phase into separate dimensions
    magnitude = tf.math.abs(fft)
    phase = tf.math.angle(fft)
    fft = tf.concat([magnitude, phase], axis=-1)
    
    return fft, resized

In [8]:
# Use tf.data.Datasets to preprocess and iterate data efficiently

batch_size = 16

train_dataset = tf.data.Dataset.from_tensor_slices(x_train)
train_dataset = train_dataset.map(cifar_parser)
train_dataset = train_dataset.shuffle(buffer_size=4096)
train_dataset = train_dataset.batch(batch_size)

test_dataset = tf.data.Dataset.from_tensor_slices(x_test)
test_dataset = test_dataset.map(cifar_parser)
test_dataset = test_dataset.batch(batch_size)

dev_dataset = tf.data.Dataset.from_tensor_slices(x_dev)
dev_dataset = dev_dataset.map(cifar_parser)
dev_dataset = dev_dataset.shuffle(buffer_size=128)
dev_dataset = dev_dataset.batch(batch_size)

# Model

In [9]:
# A metric to use during training
def mean_PSNR(y_true, y_pred):
    max_value = 1.0
    MSE = tf.reduce_mean(tf.square(y_true - y_pred), axis=[1, 2, 3])
    PSNR = 10 * tf.math.log(tf.divide(max_value ** 2, MSE)) / tf.math.log(tf.constant(10, dtype=y_pred.dtype))
    mean = tf.reduce_mean(PSNR)
    return mean

In [24]:
def load_uncompiled_automap_model():
    
    # this one's solid, but I believe we'll need a few hours to train it.
    
    N = 256
    F = N // 2
    X = tf.keras.layers.Input(shape=(N, N, 2))

    # Half-assed data augmentation
    noisy_X = tf.keras.layers.GaussianNoise(stddev=1e-7)(X) # shape: (256, 256, 256)

    # These layers all halve the spatial dimension (but also each output 256 channels)
    conv1 = tf.keras.layers.Conv2D(F, (3, 3), strides=(1, 1), activation='relu', padding='same')(noisy_X)
    pool1 = tf.keras.layers.AveragePooling2D(pool_size=2)(conv1) # shape: (128, 128, F)

    conv2 = tf.keras.layers.Conv2D(F, (3, 3), strides=(1, 1), activation='relu', padding='same')(pool1)
    pool2 = tf.keras.layers.AveragePooling2D(pool_size=2)(conv2) # shape: (64, 64, F)

    conv3 = tf.keras.layers.Conv2D(F, (3, 3), strides=(1, 1), activation='relu', padding='same')(pool2)
    pool3 = tf.keras.layers.AveragePooling2D(pool_size=2)(conv3) # shape: (32, 32, F)

    conv4 = tf.keras.layers.Conv2D(F, (3, 3), strides=(1, 1), activation='relu', padding='same')(pool3)
    pool4 = tf.keras.layers.AveragePooling2D(pool_size=2)(conv4) # shape: (16, 16, F)

    conv5 = tf.keras.layers.Conv2D(F, (3, 3), strides=(1, 1), activation='relu', padding='same')(pool4)
    pool5 = tf.keras.layers.AveragePooling2D(pool_size=2)(conv5) # shape: (8, 8, F)

    conv6 = tf.keras.layers.Conv2D(F, (3, 3), strides=(1, 1), activation='relu', padding='same')(pool5)
    pool6 = tf.keras.layers.AveragePooling2D(pool_size=2)(conv6) # shape: (4, 4, F)

    # A "FC-like" layer for fun before we do upsampling
    conv7 = tf.keras.layers.Conv2D(F, (1, 1), strides=(1, 1), activation='relu', padding='same')(pool6) # spatial dim: 4

    # These transposed convolutions upsample spatial dimension by 2
    t_conv1 = tf.keras.layers.Conv2DTranspose(F, 4, strides=2, activation='relu', padding='same')(conv7) # spatial dim: 8
    t_conv2 = tf.keras.layers.Conv2DTranspose(F, 4, strides=2, activation='relu', padding='same')(t_conv1) # spatial dim: 16
    t_conv3 = tf.keras.layers.Conv2DTranspose(F, 4, strides=2, activation='relu', padding='same')(t_conv2) # spatial dim: 32
    t_conv4 = tf.keras.layers.Conv2DTranspose(F, 4, strides=2, activation='relu', padding='same')(t_conv3) # spatial dim: 64
    t_conv5 = tf.keras.layers.Conv2DTranspose(F, 4, strides=2, activation='relu', padding='same')(t_conv4) # spatial dim: 128
    
    Y_pred = tf.keras.layers.Conv2DTranspose(1, 4, strides=2, activation='linear', padding='same')(t_conv5) # spatial dim: 256

    model = tf.keras.Model(inputs=X, outputs=Y_pred)

    return model


def load_compiled_automap_model():
    model = load_uncompiled_automap_model()
    model.compile(loss='mse', optimizer=tf.keras.optimizers.Adam(), metrics=[mean_PSNR])
    return model

In [25]:
# Custom learning rate schedule


def lr_schedule(epoch):
    """
    Returns a custom learning rate that decreases as epochs progress.
    """
    learning_rate = 5e-4
    if epoch > 3:
        learning_rate = 1e-4
    elif epoch > 6:
        learning_rate = 1e-5
    elif epoch > 10:
        learning_rate = 5e-5
    elif epoch > 15:
        learning_rate = 1e-5
    elif epoch > 20:
        learning_rate = 1e-6
    elif epoch > 40:
        learning_rate = 1e-5
    elif epoch > 80:
        learning_rate = 5e-5
    elif epoch > 100:
        learning_rate = 1e-6

    with file_writer.as_default():
        tf.summary.scalar('learning rate', data=learning_rate, step=epoch)
        
    return learning_rate

# Show reconstructions during training

def plot_fft_reconstructions(batch, logs):
    plot_frequency = 100
    
    if batch % plot_frequency != 0:
        return
    
    x, y = next(iter(test_dataset)) # always gets the first batch
    y = y.numpy()
    y_pred = model.predict(x)
    
    y = np.reshape(y, (-1, 256, 256, 1))
    y_pred = np.reshape(y_pred, (-1, 256, 256, 1))
    
    with file_writer.as_default():
        for i in range(min(len(y), 8)):
            prediction, ground_truth = y_pred[i:i + 1, ...], y[i:i + 1, ...]
            tf.summary.image("Test Image {} (Prediction)".format(i), prediction, max_outputs=1, step=batch)
            tf.summary.image("Test Image {} (Ground Truth)".format(i), ground_truth, max_outputs=1, step=batch)


In [26]:
model = load_compiled_automap_model()

In [27]:
model.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 256, 256, 2)]     0         
_________________________________________________________________
gaussian_noise_2 (GaussianNo (None, 256, 256, 2)       0         
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 256, 256, 128)     2432      
_________________________________________________________________
average_pooling2d_12 (Averag (None, 128, 128, 128)     0         
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 128, 128, 128)     147584    
_________________________________________________________________
average_pooling2d_13 (Averag (None, 64, 64, 128)       0         
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 64, 64, 128)       1475

# Training

In [28]:
# Define where logs will be saved

logdir = "logs/scalars/" + datetime.now().strftime("%Y-%m-%d-%H-%M-%S")
file_writer = tf.summary.create_file_writer(logdir + "/metrics")

In [29]:
# Create callbacks to use in various stages of training

plot_images_callback = tf.keras.callbacks.LambdaCallback(on_batch_end=plot_fft_reconstructions)
lr_callback = tf.keras.callbacks.LearningRateScheduler(lr_schedule)
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir, histogram_freq=5, update_freq=500,
                                                      profile_batch=0) # workaround for: https://github.com/tensorflow/tensorboard/issues/2084

callbacks = [tensorboard_callback, lr_callback, plot_images_callback]

In [ ]:
training_history = model.fit(
    dev_dataset, # change to train_dataset!
    validation_data=test_dataset,
    verbose=1, # set to 0 to suppress chatty output and use Tensorboard instead
    epochs=120,
    callbacks=callbacks)

Epoch 1/150
  1/313 [..............................] - ETA: 24:42 - loss: 0.3203 - mean_PSNR: 5.3693

W0529 09:03:21.623058 139627066590976 callbacks.py:257] Method (on_train_batch_end) is slow compared to the batch update (0.409698). Check your callbacks.


312/313 [============================>.] - ETA: 0s - loss: 0.0710 - mean_PSNR: 14.5535

In [ ]:
# # # Uncomment to save model
# saved_model_path = 'automap_cifar100'
# model.save(saved_model_path)

In [ ]:
plt.figure()
plt.plot(training_history.history["loss"], label="Train")
plt.plot(training_history.history["val_loss"], label="Test")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend(loc="center right")

In [ ]:
plt.figure()
plt.plot(training_history.history["mean_PSNR"], label="Train")
plt.plot(training_history.history["val_mean_PSNR"], label="Test")
plt.xlabel("Epoch")
plt.ylabel("µ ( PSNR ) ")
plt.legend(loc="center right")

In [ ]:
model.evaluate(test_dataset)

In [ ]:
model.evaluate(dev_dataset)

In [ ]:
model.evaluate(train_dataset)

In [ ]:
# Predict on a test batch

for batch in test_dataset:
    x, y = x.numpy(), y.numpy()
    y_pred = model.predict(x)
    break

In [ ]:
# Inspect output

for i in range(len(x)):
    
    fft_mag = x[i, ..., 0]
    fft_ang = x[i, ..., 1]
    c = int(cls[i])
    reconstruction = y_pred[i, ..., 0]
    reconstruction[reconstruction < 0] = 0
    reconstruction[reconstruction > 1] = 1
    image = y[i, ..., 0]

    print('Class: {}'.format(c))
    
    MSE = utils.signal_processing.mean_square_error(reconstruction, image)
    PSNR = utils.signal_processing.PSNR(reconstruction, image, max_value=1.0)

    plt.subplot(2, 2, 1)
    plt.title('Reconstruction (MSE: {:0.5f}, PSNR: {:0.5f})'.format(MSE, PSNR))
    utils.plot.imshowgray(reconstruction)

    plt.subplot(2, 2, 2)
    plt.title('FFT (Magnitude)')
    utils.plot.imshowfft(fft_mag)

    plt.subplot(2, 2, 3)
    plt.title('Expected reconstruction')
    utils.plot.imshowgray(image)

    plt.subplot(2, 2, 4)
    plt.title('FFT (Phase)')
    utils.plot.imshowgray(fft_ang)

    plt.show()
    
    break

In [ ]:
# # In theory, in TF 2.0 we should be able to see Tensorboard in this notebook with magics:
# %load_ext tensorboard
# %tensorboard --logdir logs

# Clear logs if needed
# !rm -rf logs/

In [ ]:
# !wget https://www.dropbox.com/s/1l4z7u062nvlhrz/MRI_Kspace.dat
# See https://github.com/kmjohnson3/ML4MI_BootCamp/blob/fe9d96cd9f68db073a44f9dc9a015533a008d0a7/ImageReconstruction/CoLab_AutoMap_Recon.ipynb

In [ ]:
ll -h MRI_Kspace.dat

In [ ]:
# def load_uncompiled_automap_model():
#     N = 256
#     small_N = N // 4 # after downsampling by 2 twice
    
#     X = tf.keras.layers.Input(shape=(N, N, 2))
#     noisy_X = tf.keras.layers.GaussianNoise(stddev=0.001)(X)
#     conv_downsample1 = tf.keras.layers.Conv2D(3, (4, 4), strides=(2, 2), activation='tanh', padding='same')(noisy_X)
#     conv_downsample2 = tf.keras.layers.Conv2D(3, (4, 4), strides=(1, 1), activation='tanh', padding='same')(conv_downsample1)
#     conv_downsample3 = tf.keras.layers.Conv2D(3, (4, 4), strides=(2, 2), activation='tanh', padding='same')(conv_downsample2)
#     X1 = tf.keras.layers.Flatten()(conv_downsample3)
    
#     # Workaround for: ValueError: The last dimension of the inputs to `Dense` should be defined. Found `None`.
#     X1_reshaped = tf.keras.layers.Reshape(target_shape=((small_N ** 2) * 2,))(X1)
#     X1_DO = tf.keras.layers.Dropout(0.25)(X1_reshaped)
    
#     fc1 = tf.keras.layers.Dense((small_N ** 2) * 1, activation = 'tanh')(X1_DO)
#     fc1_DO = tf.keras.layers.Dropout(0.2)(fc1)
    
#     fc2 = tf.keras.layers.Dense(small_N ** 2, activation = 'tanh')(fc1_DO)
#     fc2_DO = tf.keras.layers.Dropout(0.2)(fc2)
#     fc3 = tf.keras.layers.Dense(small_N ** 2, activation = 'tanh')(fc2_DO)

# #     fc3 = tf.keras.layers.Dense(small_N ** 2, activation = 'tanh')(fc1_DO)
#     X2 = tf.keras.layers.Reshape((small_N, small_N, 1))(fc3)
    
#     conv1_1 = tf.keras.layers.Conv2D(small_N, 5, activation='relu', padding='same', kernel_regularizer=tf.keras.regularizers.l1(0.0001))(X2)
#     conv1_2 = tf.keras.layers.Conv2D(small_N, 5, activation='relu', padding='same', kernel_regularizer=tf.keras.regularizers.l1(0.0001))(conv1_1)
#     conv1_3a = tf.keras.layers.Conv2DTranspose(small_N, 9, activation='relu', padding='same')(conv1_2)
#     conv1_3b = tf.keras.layers.Conv2DTranspose(small_N, 9, strides=2, activation='relu', padding='same')(conv1_3a)
#     conv1_3c = tf.keras.layers.Conv2DTranspose(small_N, 9, strides=2, activation='relu', padding='same')(conv1_3b)
    
#     Y_pred = tf.keras.layers.Conv2D(1, 1, activation = 'linear',padding='same')(conv1_3c)
    
#     model = tf.keras.Model(inputs=X, outputs=Y_pred)
    
#     return model
